In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

In [3]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.drop = nn.Dropout2d(0.1)

        self.conv1 = nn.Conv2d(1, 16, 3, padding=1, bias=False)        #(-1,28,28,3)>(-1,3,3,3,16)>(-1,28,28,16)
        self.batchnorm1 = nn.BatchNorm2d(16)                           #(-1,28,28,16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1, bias=False)       #(-1,28,28,16)>(-1,3,3,16,16)>(-1,28,28,16)
        self.batchnorm2 = nn.BatchNorm2d(16)                           #(-1,28,28,16)
        
        self.pool1 = nn.MaxPool2d(2, 2)                                #(-1,14,14,16)
        
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1, bias=False)       #(-1,14,14,16)>(-1,3,3,16,16)>(-1,14,14,16)
        self.batchnorm3 = nn.BatchNorm2d(16)                           #(-1,14,14,16)
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1, bias=False)       #(-1,14,14,16)>(-1,3,3,16,16)>(-1,14,14,16)
        self.batchnorm4 = nn.BatchNorm2d(16)                           #(-1,14,14,16)
        
        self.pool2 = nn.MaxPool2d(2, 2)                                #(-1,7,7,16)
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1, bias=False)       #(-1,7,7,16)>(-1,3,3,16,32)>(-1,7,7,32)
        self.batchnorm5 = nn.BatchNorm2d(32)        
        self.conv6 = nn.Conv2d(32, 16, 3, padding=1, bias=False)       #(-1,7,7,32)>(-1,3,3,32,16)>(-1,7,7,16)
        self.batchnorm6 = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16, 10, 3)                              #(-1,7,7,16)>(-1,3,3,16,10)>(-1,5,5,10)
        self.avgpool = nn.AvgPool2d(5)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.drop(x)
        x = self.batchnorm1(x)
        x = F.relu(self.conv2(x))
        x = self.drop(x)
        x = self.batchnorm2(x)
        
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = self.drop(x)
        x = self.batchnorm3(x)
        x = F.relu(self.conv4(x))
        x = self.drop(x)
        x = self.batchnorm4(x)
        
        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.drop(x)
        x = self.batchnorm5(x)

        x = F.relu(self.conv6(x))
        x = self.drop(x)
        x = self.batchnorm6(x)
        
        x = self.conv7(x)
        x = self.avgpool(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
         Dropout2d-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 16, 28, 28]           2,304
         Dropout2d-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,304
         Dropout2d-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,304
        Dropout2d-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []

from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc.append(100. * correct / len(test_loader.dataset))

In [7]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 20
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.19349819421768188 batch_id=468 Accuracy=80.86: 100%|██████████| 469/469 [00:23<00:00, 19.64it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0835, Accuracy: 9773/10000 (97.730%)

EPOCH: 1


loss=0.06665188819169998 batch_id=468 Accuracy=95.45: 100%|██████████| 469/469 [00:23<00:00, 19.74it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0195, Accuracy: 9934/10000 (99.340%)

EPOCH: 18


loss=0.04188084229826927 batch_id=468 Accuracy=98.79: 100%|██████████| 469/469 [00:23<00:00, 19.67it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0205, Accuracy: 9929/10000 (99.290%)

EPOCH: 19


loss=0.06225403770804405 batch_id=468 Accuracy=98.79: 100%|██████████| 469/469 [00:23<00:00, 19.68it/s]



Test set: Average loss: 0.0198, Accuracy: 9939/10000 (99.390%)

